## Iterative Cleaning of the Training and Test Data

In [209]:
#imports
import pandas as pd
import numpy as np
import missingno as msn
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [210]:
#load the datasets
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

## DataFrame Cleaner
Certain functions will need to be done to every DataFrame. This function will help streamline that process.

In [211]:
def df_cleaner(df):
    '''
    Returns a cleaned version of the DataFrame passed.
    Columns Checked: overall_qual, gr_liv_area, year_built, year_remod/add
    Columns Cleaned: garage_area, total_bsmt_sf, bsmt_full_bath, full_bath, bsmt_half_bath, half_bath
    '''
    #rename columns pythonically
    df.columns = [col.lower().replace(' ', '_') for col in df.columns]
    
    ##numeric columns
    #check columns that can't be 0 or null for null values
    for col in ['overall_qual', 'gr_liv_area', 'year_built', 'year_remod/add', 'lot_config', 'paved_drive', 'neighborhood', 'lot_frontage', 'exter_qual']:
        if df[col].isna().sum():
            print(f"{col.title()} has {df[col].isna().sum()} null values")
        
    #check for values outside the appropriate range
    if set(df['overall_qual'].unique()) - set(range(1,11)):
        print(f"Overall_qual has outlier {set(df['overall_qual'].unique()) - set(range(1,11))}")
    
    #replace potential NaNs in appropriate numeric columns with 0
    for col in ['garage_area', 'total_bsmt_sf', 'bsmt_full_bath', 'full_bath', 'bsmt_half_bath', 'half_bath']:
        df[col] = df[col].replace(np.nan, 0)

    ##categorical columns
    #make privacy fence column - GdPrv
    df['privacy_fence'] = df['fence'].map(lambda x: 1 if 'GdPrv' else 0)
    
    #check categoricals for outliers
    if set(df['lot_config'].unique()) - {'Inside', 'Corner', 'CulDSac', 'FR2', 'FR3'}:
        print(f"Lot_config has outlier {set(df['lot_config'].unique()) - {'Inside', 'Corner', 'CulDSac', 'FR2', 'FR3'}}")
    
    if set(df['paved_drive'].unique()) - {'Y', 'P', 'N'}:
        print(f"Paved_drive has outlier {set(df['paved_drive'].unique()) - {'Y', 'P', 'N'}}")

        
    ##ordinal columns
    #exter_qual
    if set(df['exter_qual'].unique()) - {'Ex', 'Gd', 'TA', 'Fa', 'Po'}:
        print(f"Exter_qual has outlier {set(df['exter_qual'].unique()) - {'Ex', 'Gd', 'TA', 'Fa', 'Po'}}")
    df['exter_qual_ord'] = df['exter_qual'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1})
    
    
        
#bsmt_qual
#kitchen_qual
    
    #Functional?
    
    #condition?
    
    #zoning
    if set(df['ms_zoning'].unique()) - {'A (agr)', 'C (all)', 'FV', 'I (all)', 'RH', 'RL', 'RP', 'RM'}:
        print(f"MS Zoning has outlier {set(df['ms_zoning'].unique()) - {'A', 'C', 'FV', 'I', 'RH', 'RL', 'RP', 'RM'}}")
    
    #neighborhood?
    ngbr_difference = set(df['neighborhood'].unique()) - {'Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr', 'Crawfor', 'Edwards', 'Gilbert', 'Greens', 'GrnHill', 'IDOTRR', 'Landmrk', 'MeadowV', 'Mitchel', 'NAmes', 'NoRidge', 'NPkVill', 'NridgHt', 'NWAmes', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW', 'Somerst', 'StoneBr', 'Timber', 'Veenker'}
    if ngbr_difference:
        print(f"Neighborhood has outlier {ngbr_difference}")
        

    ##feature engineering
    df['years_interact'] = df['year_built'] * df['year_remod/add']
    df['sf_interact'] = df['gr_liv_area'] * df['total_bsmt_sf']
    df['total_full_bath'] = df['bsmt_full_bath'] + df['full_bath']
    
    return df

## Training Specific Cleaning:

In [218]:
train = df_cleaner(train)
test = df_cleaner(test)

Lot_Frontage has 330 null values
Lot_Frontage has 160 null values


In [219]:
#drop training basement outliers:
train = train.drop(train[train['total_bsmt_sf'] > 4000].index)

#drop frontage outliers
train = train.drop(train[train['lot_frontage'] > 200].index)

#drop lot area outliers
train = train.drop(train[train['lot_area'] > 100000].index)

In [220]:
#save cleaned data to new csvs
train.to_csv('../data/cleaned_train.csv', index=False)
test.to_csv('../data/cleaned_test.csv', index=False)

## Explore the Data
Because there are so many variables here, first I will explore some preliminary variables of interest and then choose some to clean.

In [ ]:
plt.figure(figsize=(5,7))
sns.heatmap(train.corr()[['saleprice']].sort_values(by='saleprice', key=abs, ascending=False).head(10), annot=True, vmin=-1, vmax=1)

In [ ]:
train.describe().T

In [ ]:
#vif stuff: https://corporatefinanceinstitute.com/resources/knowledge/other/variance-inflation-factor-vif/
#adapted from vif helper code written by Devin
def vif_df(data):
    data = data.dropna()
    df = pd.DataFrame(columns=['vif'], index=data.columns)
    df['vif'] = [variance_inflation_factor(data.values, i) for i in range(len(data.columns))]
    
    return df.sort_values(by='vif', ascending=False)

In [ ]:
vif_1 = vif_df(train._get_numeric_data())
vif_1

In [ ]:
vif_2 = vif_df(train._get_numeric_data().drop(columns=[x for x in vif_1.index if vif_1.loc[x, 'vif'] > 50000]))
vif_2

In [ ]:
vif_df(train[vif_2.index].drop(columns=['year_built', 'year_remod/add', 'id']))

In [ ]:
variance_inflation_factor(train[['gr_liv_area', 'total_bsmt_sf']].values, 1)

In [ ]:
plt.scatter(train['gr_liv_area'], train['total_bsmt_sf'])

In [ ]:
plt.scatter(train['gr_liv_area'] * train['total_bsmt_sf'], np.log(train['saleprice']))

In [ ]:
plt.scatter(train['gr_liv_area'] * train['total_bsmt_sf'], train['gr_liv_area'])

In [213]:
variance_inflation_factor(train[['gr_liv_area', 'overall_cond']].values, 1)

KeyError: "None of [Index(['gr_liv_area', 'overall_cond'], dtype='object')] are in the [columns]"

In [ ]:
test.columns

In [ ]:
train['sale_type'].unique()

In [ ]:
train[train.corr()[['saleprice']].sort_values(by='saleprice', key=abs, ascending=False).head(10).index].info()

In [ ]:
test.isna().sum().sum()

In [ ]:
train[train['gr_liv_area'] > 4000]

In [ ]:
#look for suspicious min and max values
for col in train.corr()[['saleprice']].sort_values(by='saleprice', key=abs, ascending=False).head(10).index:
    print(f"{col.title()} min/Max: {train[col].min()} / {train[col].max()}")
    
    

In [ ]:
train.isna().sum().sort_values(ascending=False).iloc[:26]

In [214]:
train.loc[1280]

Id                   1743
PID             528228275
MS SubClass           120
MS Zoning              RL
Lot Frontage         53.0
                  ...    
Misc Val                0
Mo Sold                 2
Yr Sold              2007
Sale Type             New
SalePrice          194201
Name: 1280, Length: 81, dtype: object

In [215]:
plt.figure(figsize=(5,15))
sns.heatmap(train.corr()[['saleprice']].sort_values(by='saleprice', key=abs, ascending=False), annot=True, vmin=-1, vmax=1)

KeyError: "None of [Index(['saleprice'], dtype='object')] are in the [columns]"

<Figure size 360x1080 with 0 Axes>

In [ ]:
train['gr_liv_area'].min(), train['gr_liv_area'].max()

In [ ]:
test.info()

In [ ]:
train[train['total_bsmt_sf'] > 5000]

In [ ]:
train['total_bsmt_sf'].min(), train['total_bsmt_sf'].max()

In [ ]:
train[train['total_bsmt_sf'] == 0].loc[:,[x for x in train.columns if 'bsmt' in x]]

In [ ]:
test.isna().sum().sort_values(ascending=False).iloc[:26]

In [ ]:
train.dropna(axis=0, thresh=train.shape[1] - 5).shape, train.shape

In [ ]:
msn.matrix(train)

In [ ]:
plt.hist(train['saleprice'], bins=50);
plt.axvline(train['saleprice'].mean(), color='red')

In [216]:
plt.hist(np.log(train['saleprice']), bins=50)
plt.axvline(np.log(train['saleprice']).mean(), color='red')
plt.show()

KeyError: 'saleprice'

In [ ]:
sns.pairplot(train, x_vars=['garage_area', 'total_bsmt_sf', 'bsmt_full_bath', 'full_bath', 'bsmt_half_bath', 'half_bath','overall_qual', 'gr_liv_area', 'year_built', 'year_remod/add', 'lot_frontage', 'lot_area'], y_vars=['saleprice'])


In [ ]:
plt.scatter(train['lot_frontage'], train['lot_area'])

In [ ]:
plt.scatter(train['lot_frontage'] * train['lot_area'], train['saleprice'])

In [ ]:
plt.hist(train[train['year_built'] > 1995]['saleprice'], bins=50)
plt.axvline(train[train['year_built'] > 1995]['saleprice'].mean(), color='red')

In [ ]:
test[test['lot_area'] > 100000]

In [ ]:
test[test['gr_liv_area'] > 4000]

In [ ]:
plt.scatter(train['lot_frontage'] * train['lot_area'], train['saleprice'])

In [ ]:
plt.scatter(train[train['year_built'] <= 2002]['year_built'], train[train['year_built'] <= 2002]['saleprice'])

In [ ]:
plt.scatter(train[train['year_built'] > 2002]['year_built'], train[train['year_built'] > 2002]['saleprice'])

In [ ]:
train[train['lot_frontage'].isna()]

In [217]:
train[train['lot_frontage'].isna()]

KeyError: 'lot_frontage'

#### Categorical Variables

In [ ]:
dummy_df = pd.get_dummies(train, columns=[x for x in train.columns if x not in train._get_numeric_data().columns]).drop(columns=[x for x in train._get_numeric_data().columns if x != 'saleprice'])
dummy_df.describe().T

In [ ]:
sns.heatmap(dummy_df.corr()[['saleprice']].sort_values(by='saleprice', key=abs, ascending=False).head(15), annot=True, vmin=-1, vmax=1)



In [ ]:
#exter_qual
#bsmt_qual
#kitchen_qual

### Feature Engineering

I quickly noticed that when looking at the scatterplots of Year Built and Year Remodeled that there was a curve in the plot. This, combined with the realization above of their multi-colinearity led me to create an interaction term. The flaring in the later years should be accounted for with the log transformation of Sale Price.

In [ ]:
fig, ax = plt.subplots(1, 3, sharey='all', figsize=(16, 5))

for x, col in enumerate([x for x in train.columns if 'year' in x]):
    ax[x].scatter(train[col], np.log(train['saleprice']))
    ax[x].set_xlabel(col)
    
ax[0].set_ylabel('Sale Price')


In [ ]:
train[train['year_remod/add'] <= 1950]